In [1]:
# Importando Bibliotecas
import os
import torch
import matplotlib.pyplot as plt
from torchvision.datasets import CIFAR10

In [2]:
DATA_DIR = './'
CATEGORIES = ['Avião', 'Automóvel','Passáro','Gato','Cervo','Cachorro', 'Sapo', 'Cavalo', 'Navio', 'Caminhão']

In [3]:
# Dados
cifar10_train = CIFAR10(DATA_DIR, train=True, download=True)
cifar10_test = CIFAR10(DATA_DIR, train=False, download=True)

100%|██████████| 170498071/170498071 [00:04<00:00, 37674622.88it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [4]:
(len(cifar10_train), len(cifar10_test))

(50000, 10000)

In [5]:
import torchvision.transforms as tt
prep_transform = tt.Compose([tt.ToTensor(), tt.Normalize(
    (0.4914, 0.4822, 0.4465),
    (0.2470, 0.2435, 0.2616)
)])
tensor_train = CIFAR10(DATA_DIR, train=True, download=False, transform=prep_transform)
tensor_test = CIFAR10(DATA_DIR, train=False, download=False, transform=prep_transform)

In [6]:
imgs = torch.stack([img_t for img_t, _ in tensor_train], dim=3)
imgs.shape

torch.Size([3, 32, 32, 50000])

In [7]:
imgs.view(3, -1).mean(dim=1) # Média das Cores

tensor([-1.2762e-06, -1.7074e-04,  1.1819e-04])

In [8]:
imgs.view(3, -1).std(dim=1) # Desvio Padrão das Cores

tensor([1.0001, 0.9999, 1.0000])

In [9]:
from torch.utils.data import DataLoader
batch_size = 64

train_loader = DataLoader(tensor_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(tensor_test, batch_size=batch_size, shuffle=True)

In [10]:
# Testando o MLP para classificar imagem

from torch import nn

class MLPClassifier(nn.Module) :
    def __init__(self) :
        super().__init__()

        self.flatten = nn.Flatten()

        self.layers = nn.Sequential(
            nn.Linear(32 * 32 * 3, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x) :
        v = self.flatten(x)
        return self.layers(v)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Rodando na {device}')

Rodando na cuda


In [12]:
model = MLPClassifier().to(device)

In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
lossfunc = nn.CrossEntropyLoss()

In [20]:
def train(model, dataloader, lossfunc, optimizer) :
    model.train()
    cumloss = 0.0
    for imgs, labels in dataloader :
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        pred = model(imgs)
        loss = lossfunc(pred, labels)
        loss.backward()
        optimizer.step()
        cumloss += loss.item()
    return cumloss / len(dataloader)

def validate(model, dataloader, loss_func) :
    model.eval()
    cumloss = 0.0
    with torch.no_grad() :
        for imgs, labels in dataloader :
            imgs, labels = imgs.to(device), labels.to(device)
            pred = model(imgs)
            loss = loss_func(pred, labels)
            cumloss += loss.item()
        return cumloss / len(dataloader)

In [15]:
def plot_losses(losses):
    fig = plt.figure(figsize=(13, 5))
    ax = fig.gca()
    for loss_name, loss_values in losses.items():
        ax.plot(loss_values, label=loss_name)
    ax.legend(fontsize="16")
    ax.set_xlabel("Iteração", fontsize="16")
    ax.set_ylabel("Perda", fontsize="16")
    ax.set_title("Perdas vs Iterações", fontsize="16")

In [ ]:
epochs = 50
train_losses = []
test_losses = []
for t in range(epochs) :
    train_loss = train(model, train_loader, lossfunc, optimizer)
    train_losses.append(train_loss)
    test_loss = validate(model, test_loader, lossfunc)
    test_losses.append(test_loss)
    if(t % 10 == 0) :
        print(f"[Época][{t}] - Erro de Treinamento : {train_loss} - Erro de Teste : {test_loss}")



[Época][0] - Erro de Treinamento : 2.192463605605123 - Erro de Teste : 2.1642932557755974
[Época][10] - Erro de Treinamento : 1.8047673784558425 - Erro de Teste : 1.7845589355298668
[Época][20] - Erro de Treinamento : 1.6153887469140464 - Erro de Teste : 1.608470382204481
[Época][30] - Erro de Treinamento : 1.494816214379752 - Erro de Teste : 1.5118259707833552
[Época][40] - Erro de Treinamento : 1.3902423223266211 - Erro de Teste : 1.4408501546094372


In [ ]:
losses = {"Perda no Treinamento : " :  train_losses, "Perda no teste : " : test_losses}
plot_losses(losses)

In [ ]:
import sysconfig
from PIL import Image
img = Image.open('./Fusca.jpg')

In [ ]:
prep_transforms = tt.Compose([
    tt.Resize((32,32)),
    tt.ToTensor(),
    tt.Normalize(
    (0.4914, 0.4822, 0.4465),
    (0.2470, 0.2435, 0.2616))

])

img_tensor = prep_transforms(img)

In [ ]:
plt.imshow(img_tensor.permute(1,2, 0))

In [ ]:
batch = img_tensor.unsqueeze(0).to(device)

In [ ]:
import operator
output = model(batch)
logits = torch.nn.functional.softmax(output, dim=1) * 100
prob_dict = {}



for i, classname in enumerate(CATEGORIES):
  prob = logits[0][i].item()
  prob_dict[classname] = [prob]

result = max(prob_dict.items(), key=operator.itemgetter(1))
result